In [4]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

import pickle
import numpy as np

In [5]:
pickle_in = open("X_HANDWRITTEN.pickle","rb")
X = pickle.load(pickle_in)


pickle_in = open("y_HANDWRITTEN.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

model = tf.keras.models.Sequential([
   tf.keras.layers.Conv2D(input_shape=(100,100,1),filters = 32,kernel_size=(3,3),strides = (1,1),data_format='channels_last',activation = 'relu'),
   tf.keras.layers.BatchNormalization(),
   tf.keras.layers.MaxPool2D(),
  
   tf.keras.layers.Conv2D(filters = 64,kernel_size=(3,3),data_format = 'channels_last',activation='relu'),
   tf.keras.layers.BatchNormalization(),
   tf.keras.layers.MaxPool2D(),
   
   tf.keras.layers.Conv2D(filters = 128,kernel_size=(3,3),strides = (1,1),data_format='channels_last',activation = 'relu'),
   tf.keras.layers.BatchNormalization(), 
   tf.keras.layers.MaxPool2D(),
   
   tf.keras.layers.Conv2D(filters = 128,kernel_size=(3,3),data_format = 'channels_last',activation = 'relu'),
   tf.keras.layers.BatchNormalization(),
   tf.keras.layers.MaxPool2D(),
  
   tf.keras.layers.Conv2D(filters = 256,kernel_size=(3,3),data_format = 'channels_last',activation = 'relu'),
   tf.keras.layers.BatchNormalization(),
   tf.keras.layers.MaxPool2D(),
   
   tf.keras.layers.Flatten(data_format = 'channels_last'),
    
   tf.keras.layers.Dense(1024,activation = 'relu',kernel_regularizer = tf.keras.regularizers.l2(l = 0.09)),
   tf.keras.layers.BatchNormalization(),
    
   tf.keras.layers.Dense(256,activation = 'relu',kernel_regularizer = tf.keras.regularizers.l2(l = 0.09)),
   tf.keras.layers.BatchNormalization(),
    
   tf.keras.layers.Dense(16,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=16, epochs=10, validation_split=0.25)

model.save('CNNHANDWRITTEN.model')
model.save_weights('CNNHANDWRITTENWEIGHTS.h5')

Train on 1326 samples, validate on 442 samples
Epoch 1/10
1326/1326 [==============================] - 28s 21ms/sample - loss: 41.9324 - accuracy: 0.5686 - val_loss: 19.3632 - val_accuracy: 0.0679
Epoch 2/10
1326/1326 [==============================] - 27s 20ms/sample - loss: 8.8897 - accuracy: 0.7624 - val_loss: 6.8924 - val_accuracy: 0.0724
Epoch 3/10
1326/1326 [==============================] - 26s 20ms/sample - loss: 3.0240 - accuracy: 0.8062 - val_loss: 6.0244 - val_accuracy: 0.0566
Epoch 4/10
1326/1326 [==============================] - 29s 22ms/sample - loss: 1.8281 - accuracy: 0.8386 - val_loss: 4.6549 - val_accuracy: 0.0769
Epoch 5/10
1326/1326 [==============================] - 27s 21ms/sample - loss: 1.5344 - accuracy: 0.8446 - val_loss: 7.0288 - val_accuracy: 0.0679
Epoch 6/10
1326/1326 [==============================] - 26s 20ms/sample - loss: 1.3111 - accuracy: 0.8756 - val_loss: 4.7896 - val_accuracy: 0.1878
Epoch 7/10
1326/1326 [==============================] - 26s 20m

W0112 23:55:59.655883 19876 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [9]:
def prepare(filepath):
    IMG_SIZE = 100  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    img_array=img_array/255.0  # filepathread in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1,IMG_SIZE, IMG_SIZE,1)

In [ ]:
count = 0
model = tf.keras.models.load_model("CNNHANDWRITTEN.model")
for i in range(1,104):
    prediction = model.predict([prepare("Handwritten using motion tracking/0/" + str(i) + ".jpg")])
    prediction=np.argmax(prediction)
    if(prediction!='B'):
        count = count + 1
        #print(str(i) + "-prediction here is " + str(prediction))
print("count is " + str(count))